In [3]:
import os
import torch
from torchvision.datasets.utils import download_url
# import zipfile 

import pandas as pd

### Load data

In [63]:
# Read data - short version 3400+ documents
reuters = pd.read_pickle('input/reuters_small.pkl')
print(len(reuters))
reuters[0:5]

3426


,codes,headline,text,classes
0,"[C18, C181, CCAT]",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia...","[25, 26, 44]"
1,"[G15, GCAT]",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...,"[80, 90]"
2,"[G15, GCAT]",Official Journal contents - OJ L 190 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]"
3,"[G15, GCAT]",Official Journal contents - OJ C 221 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]"
4,"[G15, GCAT]",Official Journal contents - OJ C 220 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]"


In [64]:
# load full data, if produced. 435 MB. NOT in Github currently. 
# You can produce this in about 30 min with preprocess_data.ipynb

#reuters = pd.read_pickle('input/reuters_all.pkl')

In [65]:
# read classcodes
classcodes= pd.read_csv('input/classcodes.csv')
print(len(classcodes))
#classcodes[0:12]

126


### Using DataSet


Example in:
https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/A%20-%20Using%20TorchText%20with%20Your%20Own%20Datasets.ipynb

In [66]:
from torchtext import data
from torchtext import datasets

#Define the Fields
TEXT = data.Field()
HEADLINE = data.Field()

LABELS = data.LabelField()
#CLASSES = data.LabelField()

In [67]:
reuters[0:5]

,codes,headline,text,classes
0,"[C18, C181, CCAT]",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia...","[25, 26, 44]"
1,"[G15, GCAT]",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...,"[80, 90]"
2,"[G15, GCAT]",Official Journal contents - OJ L 190 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]"
3,"[G15, GCAT]",Official Journal contents - OJ C 221 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]"
4,"[G15, GCAT]",Official Journal contents - OJ C 220 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]"


In [68]:
len(reuters)

3426

In [69]:
### Test, save pd df to JSON
                            # one record at time, one record by line
reuters.to_json('input/reuters_small.json', orient='records', lines=True)

In [70]:
# split it
train = reuters[0:2500]
test = reuters[2500:3000]
val = reuters[3000:len(reuters)]

train.to_json('input/train.json', orient='records', lines=True)
test.to_json('input/test.json', orient='records', lines=True)
val.to_json('input/val.json', orient='records', lines=True)

In [71]:
# Tell torchText which Fields to apply to which json elements

fields = {'headline': ('h', HEADLINE), 'text': ('t', TEXT), 'classes': ('l', LABELS)}

In [72]:
fields

{'headline': ('h', <torchtext.data.field.Field at 0x7f1fe4ea0cc0>),
 'text': ('t', <torchtext.data.field.Field at 0x7f1fe4ea0c88>),
 'classes': ('l', <torchtext.data.field.LabelField at 0x7f1fe4ea0cf8>)}

In [73]:
# Create dataset (TabularDataset)
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = 'input',
                                        train = 'train.json',
                                        validation = 'val.json',
                                        test = 'test.json',
                                        format = 'json',
                                        fields = fields
)

In [74]:
# test
print(vars(train_data[2]))

{'h': ['Official', 'Journal', 'contents', '-', 'OJ', 'L', '190', 'of', 'July', '19,', '1997.'], 't': ['*', '(Note', '-', 'contents', 'are', 'displayed', 'in', 'reverse', 'order', 'to', 'that', 'in', 'the', 'printed', 'Journal)', '*', 'COMMISSION', 'DECISION', 'of', '16', 'July', '1997', 'amending', 'Decision', '97/285/EC', 'concerning', 'certain', 'protection', 'measures', 'relating', 'to', 'classical', 'swine', 'fever', 'in', 'Spain', '(Text', 'with', 'EEA', 'relevance)', 'COMMISSION', 'REGULATION', '(EC)', 'No', '1399/97', 'of', '18', 'July', '1997', 'establishing', 'the', 'standard', 'import', 'values', 'for', 'determining', 'the', 'entry', 'price', 'of', 'certain', 'fruit', 'and', 'vegetables', 'COMMISSION', 'REGULATION', '(EC)', 'No', '1398/97', 'of', '18', 'July', '1997', 'fixing', 'the', 'agricultural', 'conversion', 'rates', 'COMMISSION', 'REGULATION', '(EC)', 'No', '1397/97', 'of', '18', 'July', '1997', 'fixing,', 'for', 'the', '1996/97', 'marketing', 'year,', 'the', 'specific

### continue

In [75]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train, val, test), batch_sizes=(16, 256, 256),
    sort_key=lambda x: len(x.text), device=0)


The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [76]:
TEXT.build_vocab(train)
HEADLINE.build_vocab(train)
LABELS.build_vocab(train)

In [62]:
TEXT.vocab

In [35]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 2500
Number of validation examples: 426
Number of testing examples: 500


In [37]:
TEXT.build_vocab(train_data, max_size=25000)
LABELS.build_vocab(train_data)

In [39]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABELS vocabulary: {len(LABELS.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABELS vocabulary: 95


In [40]:
print(TEXT.vocab.freqs.most_common(10))

[('the', 22529), ('of', 12791), ('to', 12004), ('and', 9792), ('in', 9599), ('a', 8831), ('on', 5491), ('said', 4417), ('for', 4282), ('The', 3677)]


In [41]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', 'of', 'to', 'and', 'in', 'a', 'on', 'said']


In [43]:
print(LABELS.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f1fe54dd0d0>, {44: 0, 15: 1, 123: 2, 16: 3, 90: 4, 70: 5, 119: 6, 18: 7, 120: 8, 116: 9, 55: 10, 114: 11, 25: 12, 26: 13, 57: 14, 13: 15, 104: 16, 115: 17, 32: 18, 33: 19, 117: 20, 118: 21, 20: 22, 91: 23, 93: 24, 46: 25, 108: 26, 64: 27, 122: 28, 11: 29, 29: 30, 80: 31, 110: 32, 56: 33, 62: 34, 100: 35, 41: 36, 21: 37, 42: 38, 66: 39, 94: 40, 38: 41, 43: 42, 121: 43, 17: 44, 12: 45, 40: 46, 45: 47, 14: 48, 22: 49, 28: 50, 97: 51, 112: 52, 48: 53, 69: 54, 87: 55, 88: 56, 99: 57, 49: 58, 35: 59, 92: 60, 27: 61, 65: 62, 111: 63, 23: 64, 113: 65, 30: 66, 84: 67, 106: 68, 107: 69, 83: 70, 105: 71, 34: 72, 47: 73, 67: 74, 82: 75, 96: 76, 19: 77, 24: 78, 58: 79, 63: 80, 31: 81, 51: 82, 59: 83, 85: 84, 103: 85, 39: 86, 50: 87, 54: 88, 81: 89, 98: 90, 36: 91, 37: 92, 68: 93, 102: 94})


In [79]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

### simple model

In [80]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.embedding(x)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [89]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [90]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [91]:
criterion = nn.BCEWithLogitsLoss()

In [92]:
model = model.to(device)
criterion = criterion.to(device)

In [93]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [94]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [95]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [96]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

TypeError: unhashable type: 'list'